In [1]:
#Sækjum gögnin sem búið er að reikna
import numpy as np

leikir_fen = np.loadtxt("leikir_fen.csv", delimiter=",", dtype=list, comments=None)
leikir_eval = np.loadtxt("leikir_eval.csv", delimiter=",", dtype=int, comments=None)
#leikir_bitmap = np.loadtxt("leikir_bitmap.csv", delimiter=",", dtype=int, comments=None)
leikir_bitmap_50k = np.loadtxt("leikir_bitmap_50k.csv", delimiter=",", dtype=int, comments=None)

In [2]:
#Lesum inn gögnin
import numpy as np

gogn = np.loadtxt("Gögn/games.csv", delimiter=",", dtype=str, comments=None)
#display(gogn)
gogn.shape

(20059, 16)

In [3]:
#Eyðum gögnum sem eru ekki notuð
dalkarTilAdEyda = [15,14,13,10,8,7,5,3,2,1]
gogn = np.transpose(gogn)
gogn = np.delete(gogn,dalkarTilAdEyda,0)
gogn = np.transpose(gogn)
gogn.shape

(20059, 6)

In [4]:
#Finnum lengd lengstu skákar
import numpy as np
max_lengd = 0
temp_max_lengd = 0
maxi = 0
for i in range(len(gogn[:,0])):
    leikir = gogn[i,5].split()
    temp_max_lengd = len(leikir)
    if temp_max_lengd > max_lengd:
        max_lengd = temp_max_lengd
        maxi = i
print(max_lengd)
print(maxi)

#Setjum skákirnar í fylki, leik fyrir leik
leikir = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(1, len(gogn[:,0])):
    temp_leikir = gogn[i,5].split()
    for j in range(len(temp_leikir)):
        leikir[i-1,j] = temp_leikir[j]
print(leikir.shape)

349
11555
(20058, 349)


In [5]:
#Breytum uppsetingu á leikjum úr (d4 d5...) í fen
import chess
leikir_fen = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(len(leikir[:,0])):
    for j in range(len(leikir[0,:])):
        if leikir[i,j] != 0:
            board = chess.Board()
            for k in range(j):
                board.push_san(leikir[i,k])
            leikir_fen[i,j] = board.fen()
        else:
            break

In [11]:
#np.savetxt("leikir_fen.csv", leikir_fen, delimiter=",", fmt="%s")

In [73]:
#Finnum evaluation fyrir hvern leik með Stockfish
from stockfish import Stockfish

stockfish = Stockfish(path="/home/magnus/Downloads/Installers of setup/stockfish_15.1_linux_x64/stockfish-ubuntu-20.04-x86-64")
stockfish.update_engine_parameters({"Threads": 10, "Minimum Thinking Time": 0, "Hash": 32})
stockfish.set_depth(10)
leikir_eval = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(len(leikir_fen[:,0])):
#for i in range(100):
    for j in range(len(leikir_fen[0,:])):
        if leikir_fen[i,j] != 0:
            stockfish.set_fen_position(leikir_fen[i,j])
            a = stockfish.get_evaluation()
            leikir_eval[i,j] = a['value']
        else:
            break

In [74]:
#np.savetxt("leikir_eval.csv", leikir_eval, delimiter=",", fmt="%s")

In [35]:
#Teljum fjölda leikja sem eru ekki 0
fjoldi_leikja = 0
for i in range(len(leikir_fen[:,0])):
    for j in range(len(leikir_fen[0,:])):
        if leikir_fen[i,j] != '0': #0 þegar keyrt fyrst en '0' þegar lesið úr skrá
            fjoldi_leikja += 1
print(fjoldi_leikja)

1212827


In [7]:
b = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
a = b[5,5]
c = leikir_fen[5,5]

In [160]:
#Breytum úr fen í bitboard og hvern leik í sér línu
leikir_bitmap = np.zeros((fjoldi_leikja, 64*12+1), dtype=int)
l = 0
for i in range(len(leikir_fen[:,0])):
    for j in range(len(leikir_fen[0,:])):
        if leikir_fen[i,j] != 0:
            board = chess.Board(leikir_fen[i,j])
            leikir_bitmap[l,0] = (j+1)//2+((j+1)%2>0) #Númer leiksins er fyrsta stakið í hverri línu
            for k in range(64):
                if board.piece_at(k) != None:
                    if board.piece_at(k).color == True:
                        leikir_bitmap[l,1+k+64*board.piece_at(k).piece_type-64] = 1
                    else:
                        leikir_bitmap[l,1+k+64*board.piece_at(k).piece_type-64+6*64] = 1
            l += 1
        else:
            break

In [166]:
#np.savetxt("leikir_bitmap.csv", leikir_bitmap, delimiter=",", fmt="%d")

In [ ]:
#leikir_bitmap_50k = leikir_bitmap[0:50000,:]
#np.savetxt("leikir_bitmap_50k.csv", leikir_bitmap_50k, delimiter=",", fmt="%d")

In [26]:
#Setjum eval í línu
leikir_eval_lina = np.zeros((fjoldi_leikja, 1), dtype=int)
l = 0
for i in range(len(leikir_eval[:,0])):
    for j in range(len(leikir_eval[0,:])):
        if leikir_fen[i,j] != '0': #Notum fen því það er ekki núll en eval getur verið núll
            leikir_eval_lina[l,0] = leikir_eval[i,j]
            l += 1
        else:
            break
leikir_eval_lina_50k = leikir_eval_lina[0:50000,:]

In [10]:
#np.savetxt("leikir_eval_lina.csv", leikir_eval_lina, delimiter=",", fmt="%d")
#np.savetxt("leikir_eval_lina_50k.csv", leikir_eval_lina_50k, delimiter=",", fmt="%d")

In [27]:
#Setjum elo í línu
leikir_elo_lina = np.zeros((fjoldi_leikja, 2), dtype=int)
l = 0
for i in range(len(leikir_eval[:,0])):
    for j in range(len(leikir_eval[0,:])):
        if leikir_fen[i,j] != '0':
            leikir_elo_lina[l,0] = gogn[i+1,3]
            leikir_elo_lina[l,1] = gogn[i+1,4]
            l += 1
leikir_elo_lina_50k = leikir_elo_lina[0:50000,:]

In [28]:
#np.savetxt("leikir_elo_lina.csv", leikir_elo_lina, delimiter=",", fmt="%d")
#np.savetxt("leikir_elo_lina_50k.csv", leikir_elo_lina_50k, delimiter=",", fmt="%d")

In [30]:
#Setjum öll gögn í eitt fylki (nr.leiks, bitboard, eval, eloHv., eloSv.)
fjoldi_leikja = 50000 #Skoða bara 50k því 1.2M er of mikið
leikir_heild = np.zeros((fjoldi_leikja, 769+1+2), dtype=int)
for i in range(fjoldi_leikja):
    leikir_heild[i,0:769] = leikir_bitmap_50k[i,:]
    leikir_heild[i,769:769+1] = leikir_eval_lina[i,:]
    leikir_heild[i,769+1:769+1+2] = leikir_elo_lina[i,:]

In [31]:
#np.savetxt("leikir_heild_50k.csv", leikir_heild, delimiter=",", fmt="%d")

In [36]:
#Setjum upp hver vann skákina í samsvarandi fylki
leikir_sigurvegari = np.zeros((fjoldi_leikja, 1), dtype=int) #1 = hvítur vann, 0 = svartur vann
l = 0
for i in range(len(leikir_fen[:,0])):
    for j in range(len(leikir_fen[0,:])):
        if leikir_fen[i,j] != '0':
            temp_leikir_sigurvegari = gogn[i+1,2]
            if temp_leikir_sigurvegari == 'white':
                leikir_sigurvegari[l,0] = 1
            l += 1
leikir_sigurvegari_50k = leikir_sigurvegari[0:50000,:]

In [37]:
#np.savetxt("leikir_sigurvegari.csv", leikir_sigurvegari, delimiter=",", fmt="%d")
#np.savetxt("leikir_sigurvegari_50k.csv", leikir_sigurvegari_50k, delimiter=",", fmt="%d")

Nú eru öll þjálfunar- og prófunargögn komin